# Chapter 7. Hypothesis and inference

## 7.1 Statistical hypothesis testing

## 7.2 Example: flipping a coin

Imagine we have a coin and we want to test whether it’s fair. We’ll make the assumption that the coin has some probability p of landing heads, and so our null hypothesis is that the coin is fair—that is, that p = 0.5. We’ll test this against the alternative hypothesis p ≠ 0.5.

In particular, our test will involve flipping the coin some number, n, times and counting the number of heads, X. Each coin flip is a Bernoulli trial, which means that X is a Binomial(n,p) random variable, which we can approximate using the normal distribution

In [1]:
from typing import Tuple
import math 

def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    '''Returns mu and sigma corresponding to a Binomial(n, p)'''
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [3]:
def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

# the normal cdf is the probability the variable is below a threshold
normal_probability_below = normal_cdf

# It's above the threshold if it is not below the threshold
def normal_probability_above(lo: float,
                             mu: float = 0,
                             sigma: float = 1) -> float:
    '''The probability that an N(mu, sigma) is greater than lo'''
    return 1 - normal_cdf(lo, mu, sigma)

# It is between if it is less than hi, but not less than lo
def normal_probability_between(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float: 
    '''The probability that an N(mu, sigma) is between lo and hi'''
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# It is outside if it is not between
def normal_probability_outside(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    '''The probability that an N(mu, sigma) is not between lo and hi'''
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [4]:
def inverse_normal_cdf(p: float,
                       mu: float = 0,
                       sigma: float = 1,
                       tolerance: float = 0.00001) -> float:
    """Find approximate inverse using binary search"""

    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)

    low_z = -10.0                      # normal_cdf(-10) is (very close to) 0
    hi_z  =  10.0                      # normal_cdf(10)  is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     # Consider the midpoint
        mid_p = normal_cdf(mid_z)      # and the cdf's value there
        if mid_p < p:
            low_z = mid_z              # Midpoint too low, search above it
        else:
            hi_z = mid_z               # Midpoint too high, search below it

    return mid_z

def normal_upper_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    '''Returns the z for which P(Z <= z) = probability'''
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    '''Returns the z for which P(Z >= z) = probability'''
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability: float,
                            mu: float = 0,
                            sigma: float = 1) -> Tuple[float, float]:
    '''Returns the symmetric (about the mean) bounds that contain the specified probability'''
    tail_probability = (1 - probability) / 2
    
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound

In [6]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

In [7]:
print(mu_0, sigma_0)

500.0 15.811388300841896


In [8]:
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)

In [9]:
print(lower_bound, upper_bound)

469.01026640487555 530.9897335951244


In [10]:
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

In [11]:
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

In [12]:
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
power

0.8865480012953671

In [13]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
power

0.9363794803307173

## 7.3 p-values

In [14]:
def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    '''How likely are we to see a value at least as extreme as x (in either direction) if our values are from an N(mu, sigma)'''
    if x >= mu:
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        return 2 * normal_probability_below(x, mu, sigma)

two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

## 7.4 Confidence intervals

## 7.5 p-hacking

## 7.6 Example: running an a/b test

## 7.7 Bayesian inference

## 7.8 For further exploration